In [ ]:
#-----------------------Laboratorio 09----------------------------
# Luiz Felipe Barros Alves

In [2]:
import sys, os, imp
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import lambda2nu, nu2lambda, c

C:\Users\alves\AppData\Local\Temp\ipykernel_18960\3778166921.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import sys, os, imp


In [3]:
# lumapiFile = "/opt/lumerical/v221/api/python/lumapi.py"
lumapiFile = "C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py"
lumapi = imp.load_source("lumapi", lumapiFile)
emeApi = lumapi.MODE(hide = False)

## Especificação dos materiais e dimensões do guia

In [ ]:
# units
um = 1e-6
nm = 1e-9

In [ ]:
# Materiais 
sub_material = 'SiO2 (Glass) - Palik'
core_material = 'Si (Silicon) - Palik'

In [ ]:
# MMI x parameters
L = 57.420 * um
W = 7.0 * um

# Taper parameters
W1 = 450 * nm
W2 = 1.5 * um
L1 = 15 * um

# Waveguide parameters
wvg_width = 450 * nm
wvg_height = 220 * nm

# Waveguide length
wvg_length = 4.0 * um

# Bandwidth operation
cband = True
filename = "S_MMI_EME_2x2_%d_um" % (L / um)

# C Band simulation: 1520 - 1575 nm
# L Band simulation: 1555 - 1635 nm


if cband:
    wvlngth_start = 1.520 * um
    wvlngth_stop = 1.575 * um
else:
    wvlngth_start = 1.555 * um
    wvlngth_stop = 1.635 * um

nmbrFrqPnts = 20


## Criação da estrutura

In [ ]:
# Definindo um grupo de estrutura
emeApi.addstructuregroup()
dev_layer = "GDS_LAYER 35:4"
emeApi.set("name", dev_layer)

In [ ]:
# Adicionando um retângulo
emeApi.addrect()
emeApi.addtogroup(dev_layer)
emeApi.set("name", "box")
emeApi.set("x", 0)
emeApi.set("x span", L)
emeApi.set("y", 0)
emeApi.set("y span", W)
emeApi.set("z", wvg_height / 2)
emeApi.set("z span", wvg_height)
emeApi.set("material", material_Si)

# Configuração dos tapers
for i in range(3):
    emeApi.addobject("linear_taper")
    emeApi.set("name", f"taper port {str(i + 1)}")
    emeApi.set("thickness", wvg_height)
    emeApi.set("angle_side", 90)
    emeApi.set("width1", W1)
    emeApi.set("width2", W2)
    emeApi.set("len", L1)
    emeApi.set("material", material_Si)
    emeApi.set("x", emeApi.getnamed(f"{dev_layer}::box", "x max") + L1/2)
    emeApi.set("y", 0)
    emeApi.set("z", wvg_height / 2)

# Ajustando as posições dos tapers
# Primeiro taper
emeApi.select(f"{dev_layer}::taper port 1")
emeApi.set("width1", W1)
emeApi.set("width2", W2)
emeApi.set("x", -emeApi.getnamed(f"{dev_layer}::box", "x max") + L1/2)
emeApi.set("y", 0)

# Terceiro taper
emeApi.select(f"{dev_layer}::taper port 3")
emeApi.set("y", -W/4)
